In [17]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp
import nltk
nltk.download('wordnet')

comments_df= pd.read_csv('post_comments.csv')
docs= array(comments_df['body'])

Populating the interactive namespace from numpy and matplotlib
[nltk_data] Downloading package wordnet to C:\Users\Niharika
[nltk_data]     Raut\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
print(len(docs))
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs


7163


In [19]:
docs = docs_preprocessor(docs)

In [20]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or
more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [26]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 8273
Number of unique words after removing rare and common words: 819


In [27]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 819
Number of documents: 7163


In [28]:
from gensim.models import LdaModel
num_topics = 10
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 1min 1s


In [29]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pyLDAvis.gensim.prepare(model, corpus, dictionary)

c:\program files\python36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.460077  0.026408       1        1  19.529133
5     -0.087381  0.438759       2        1  18.062883
6     -0.076317 -0.165293       3        1  11.074309
8     -0.042851 -0.043562       4        1   8.421175
7     -0.054572 -0.097865       5        1   8.040503
0     -0.043080 -0.039757       6        1   7.457055
3     -0.036775 -0.025278       7        1   6.957062
1     -0.042307 -0.035099       8        1   6.949207
2     -0.037212 -0.028670       9        1   6.909520
9     -0.039583 -0.029643      10        1   6.599148, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
19    Default  3484.000000      look_like  3484.000000  30.0000  30.0000
56    Default  1171.000000           that  1171.000000  29.0000  29.0000
15    Default   476.000000           this   476.000000  28.0000  28.0000
75    Default   630.000000           with   630.000000  27.0000  27.0000
4     Default   588.000000           have   588.000000  26.0000  26.0000
103   Default   306.000000           what   306.000000  25.0000  25.0000
36    Default   420.000000           face   420.000000  24.0000  24.0000
72    Default   232.000000           make   232.000000  23.0000  23.0000
47    Default   276.000000           when   276.000000  22.0000  22.0000
28    Default   243.000000           they   243.000000  21.0000  21.0000
29    Default   352.000000           from   352.000000  20.0000  20.0000
152   Default   234.000000          would   234.000000  19.0000  19.0000
177   Default   219.000000           more   219.000000  18.0000  18.0000
111   Default   214.000000        because   214.000000  17.0000  17.0000
59    Default   325.000000           just   325.000000  16.0000  16.0000
186   Default   223.000000           only   223.000000  15.0000  15.0000
217   Default   172.000000           than   172.000000  14.0000  14.0000
314   Default   174.000000           girl   174.000000  13.0000  13.0000
96    Default   174.000000           time   174.000000  12.0000  12.0000
52    Default   200.000000    your_parent   200.000000  11.0000  11.0000
46    Default   176.000000          there   176.000000  10.0000  10.0000
352   Default   163.000000           year   163.000000   9.0000   9.0000
60    Default   174.000000          those   174.000000   8.0000   8.0000
456   Default   159.000000          going   159.000000   7.0000   7.0000
257   Default   162.000000           need   162.000000   6.0000   6.0000
174   Default   179.000000           will   179.000000   5.0000   5.0000
439   Default   156.000000           fuck   156.000000   4.0000   4.0000
139   Default   141.000000          where   141.000000   3.0000   3.0000
215   Default   173.000000     only_thing   173.000000   2.0000   2.0000
9     Default   150.000000          roast   150.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
139   Topic10   141.350555          where   141.541611   2.7169  -2.8237
308   Topic10   118.662987           down   118.854279   2.7166  -2.9987
380   Topic10   112.012123           take   112.209526   2.7165  -3.0564
136   Topic10    97.033981           shit    97.224747   2.7163  -3.1999
510   Topic10    82.043190           love    82.232719   2.7159  -3.3677
641   Topic10    79.357567     doesn_make    79.546677   2.7158  -3.4010
309   Topic10    76.464371  down_syndrome    76.653221   2.7158  -3.4381
106   Topic10    76.430626           also    76.621208   2.7157  -3.4386
131   Topic10    55.508743         fucked    55.696857   2.7148  -3.7584
388   Topic10    50.958973           part    51.145767   2.7146  -3.8439
141   Topic10    51.072201        getting    51.261509   2.7145  -3.8417
134   Topic10    47.624844           damn    47.812202   2.7143  -3.9116
132   Topic10    45.1102

In [35]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [36]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                 frequency

Topic 0 |---------------------

girl                 0.065
time                 0.065
enough               0.045
work                 0.033
made                 0.030
other                0.028
same                 0.024
hope                 0.024
nice                 0.024
actually             0.023
Topic 1 |---------------------

going                0.064
doesn                0.047
school               0.045
nose                 0.042
then                 0.036
well                 0.033
friend               0.032
school_shooter       0.027
high_school          0.025
another              0.024
Topic 2 |---------------------

more                 0.088
than                 0.069
could                0.051
little               0.033
more_than            0.033
before               0.032
away_from            0.031
pretty               0.030
guess                0.030
paper                0.028
Topic 3 |---------------------

what                 0.122
yea